In [1]:
%env AWS_DEFAULT_PROFILE=protecso

env: AWS_DEFAULT_PROFILE=protecso


In [2]:
import boto3
import dynamodbgeo
import uuid
dynamodb = boto3.client('dynamodb', region_name='us-east-1')

In [3]:
config = dynamodbgeo.GeoDataManagerConfiguration(dynamodb, 'news')
geoDataManager = dynamodbgeo.GeoDataManager(config)

In [4]:
# Pick a hashKeyLength appropriate to your usage
config.hashKeyLength = 5

# Use GeoTableUtil to help construct a CreateTableInput.
table_util = dynamodbgeo.GeoTableUtil(config)
create_table_input=table_util.getCreateTableRequest()

#tweaking the base table parameters as a dict
create_table_input["ProvisionedThroughput"]['ReadCapacityUnits']=5

# Use GeoTableUtil to create the table
table_util.create_table(create_table_input)

Creating table [news]
Waiting for table [news] to be created
Table created


In [ ]:
# this is an example of how to put an item in the table
PutItemInput = {
	'Item': {
			'Country': {'S': "Peru"},
			'Capital': {'S': "Lima"},
			'year': {'S': '1000'}
	},
	'ConditionExpression': "attribute_not_exists(hashKey)" # ... Anything else to pass through to `putItem`, eg ConditionExpression

}
geoDataManager.put_Point(dynamodbgeo.PutPointInput(
	dynamodbgeo.GeoPoint(36.879163, 10.243120), # latitude then latitude longitude
	 str( uuid.uuid4()), # Use this to ensure uniqueness of the hash/range pairs.
	 PutItemInput # pass the dict here
	))

{'ResponseMetadata': {'RequestId': 'G3N4753TSSO43L8DT3OA4J064FVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 24 Nov 2024 03:36:17 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'G3N4753TSSO43L8DT3OA4J064FVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [ ]:
import json 
# load base_data/maiz_exports.json
with open('base_data/maiz_exports.json') as f:
	maiz_exports_origin_data = json.load(f)

In [8]:
maiz_exports = maiz_exports_origin_data.get('news')
len(maiz_exports)

48

In [ ]:
maiz_exports
"""
[{'id': 262001428,
	'title': 'Márgenes de exportación y molienda de soja negativos',
	'text': 'A los precios de mercado en todos los commodities, trigo, soja y maíz, tanto en el disponible como los precios futuros nueva cosecha, los márgenes de exportación son negativos. Esto quiere decir que los exportadores pierden plata si analizamos los precios FOB de mercado y los precios FAS teóricos resultantes. No se contempla en este análisis algún hipotético beneficio financiero que alguna empresa en forma individual pudiera tener.\n\nVeamos lo que sucede con el precio FOB de maíz para embarque abril/mayo: en estos momentos, está cotizando a U$S 199 la tonelada (U$S/tn), equivalente a un precio FAS teórico de 165 U$S/tn, mientras que el precio del mercado en el MATBA-Rofex está cotizando a 182 U$S/tn.\n\nLa diferencia de 17 U$S/tn es el precio que debería bajar el mercado para que la exportación tenga un margen cero, ni perdida ni ganancia. Resumiendo, el precio de ventana para el maíz nueva cosecha debería ser de 165 U$S/tn, en lugar de la cotización actual de 182 U$S/tn.\n\nLa pregunta del millón es ¿cuál será el precio del maíz nuevo cuando comience la cosecha en abril o mayo? Si uno se guía por el precio FAS teórico, el precio del maíz debería ser 165 U$S/tn, pero es muy difícil poder predecir en forma tan anticipada (faltan siete meses) a cuánto estará el mercado en la realidad.\n\nMientras tanto, los productores han comenzado a vender maíz forward nueva cosecha, posición mayo 2025, con un volumen registrado de ventas de 220 mil toneladas en la segunda semana de octubre.\n\nPor lo pronto, estamos viendo que Chicago muestra tendencia muy volátil y errática. Entre agosto y principios de octubre, el maíz en Chicago posición mayo subió 17 U$S/tn, entre principios y finales de octubre el mercado perdió 10 U$S/tn.\n\nEn el balance, la ganancia neta en el mercado fue de solamente 7 U$S/tn. Hay que destacar que estas variaciones que sufrió el mercado de Chicago no se trasladaron en forma lineal y automática a las cotizaciones del maíz sobre el mercado en Argentina. Este divorcio entre los dos mercados es consecuencia de la incertidumbre que existe con respecto al volumen de la cosecha de maíz en Brasil y en Argentina.\n\nPor su parte, la cosecha de maíz en Estados Unidos está a punto de concluir, y se espera que a partir de noviembre los precios sobre Chicago comiencen a consolidar una tendencia sostenida a firme.\n\nHarina de soja\n\nEn el caso de la soja, los márgenes de molienda, resultante a partir de la elaboración del poroto de soja y la consecuente producción de aceite y harina de soja, son negativos en 25 U$S/tn.\n\nEl precio de paridad para que la industria aceitera tenga margen de molienda “cero”, es decir, que no gane ni pierda plata, es de 264 U$S/tn, mientras que los precios en el MATBA-Rofex están cotizando a 289 U$S/tn.\n\nEn concreto, este resultado negativo del margen de molienda actúa como uno de los principales factores bajistas a la hora del ingreso de la nueva cosecha.\n\nSucede que con una siembra de soja en el orden de los 18 a 19 millones de hectáreas, podremos aspirar a tener una cosecha de soja en el rango de 52 a 55 millones de toneladas.\n\nCon este volumen de producción, más el ingreso de la soja de la cosecha anterior 2024, conocido como “carry-over”, estimado en 12 millones de toneladas, llegamos a un volumen de oferta global de 64 a 67 millones de toneladas.\n\nEste es el dato clave para el mercado, con semejante volumen de producción de soja de la nueva cosecha más el stock final de la cosecha 2024, no resiste que la industria aceitera acepte arrancar la nueva cosecha con márgenes de molienda negativos.\n\nDicho de otra forma, las chances de baja en el precio de la soja nueva de 25 U$S/tn es altamente probable.',
	'summary': '',
	'url': 'https://www.lavoz.com.ar/agro/agricultura/margenes-de-exportacion-y-molienda-de-soja-negativos/',
	'image': 'https://www.lavoz.com.ar/resizer/v2/T5XZZAGDLJFPDO2JKGW3BXKL3A.jpg?quality=75&smart=true&auth=65721a6ab7f4fb89b2f3ebd19a12bd96e6fafece241add524b7ee0068f0858ac&width=1200&height=630',
	'video': None,
	'publish_date': '2024-11-02 03:01:00',
	'author': 'Pablo Adreani',
	'authors': ['Pablo Adreani'],
	'language': 'es',
	'category': 'politics',
	'source_country': 'ar'},
"""

SyntaxError: incomplete input (2271802512.py, line 2)

In [ ]:
from dotenv import load_dotenv
load_dotenv() # OPENAI_API_KEY

from openai import OpenAI
client = OpenAI()


system_prompt = '' # load text from file system_prompt.txt
with open('system_prompt.txt') as f:
	system_prompt = f.read()

user_example = ''
with open('user_example.txt') as f:
	user_example = f.read()
		
assistant_example = ''
with open('assistant_example.txt') as f:
	assistant_example = f.read()




useful_news_for_maiz_exports = []
for item in maiz_exports:
	title = item.get('title')
	# the summary is either the attribute summary if not None or the first 200 characters of 'text'
	summary = item.get('summary') or item.get('text')[:200]
	
	completion = client.chat.completions.create(
		model="gpt-4-turbo",
		messages=[
			{"role": "system", "content": system_prompt},
			{
				"role": "user",
				"content": user_example
			},
			{
				"role": "assistant",
				"content": assistant_example
			},
			{
				"role": "user",
				"content": f"<news>{title}\n{summary}</news>"
			}
		]
	)
	try:
		completion = str(completion.choices[0].message.content)
		print(completion)
		if 'si.' in completion.lower():
			item['insight'] = completion
			useful_news_for_maiz_exports.append(item)
			print('si')
	except:
		continue

Si.
Noticias sobre márgenes negativos de exportación pueden afectar directamente las decisiones de exportación agrícola de Perú al afectar la rentabilidad.
si
Si.
Este crecimiento en las exportaciones de maíz podría influir positivamente en decisiones de exportación y cultivo de maíz en Perú, considerando tendencias de mercado y demanda.
si
Si.
La caída en los precios internacionales de soja y maíz altera las variables de comercialización y puede impactar las exportaciones peruanas en términos de competitividad y estrategia de mercado.
si
Si.
La caída de precios en los cereales podría influir en las decisiones de exportación y precios de productos peruanos relacionados al maíz y al trigo.
si
No.
No.
Si.
El aumento en la molienda de soja indica una recuperación en esta industria crucial para la exportación, relevante para la planificación de ventas internacionales.
si
No.
No.
No.
No.
Si.
La falta de diésel puede afectar directamente la producción y exportación agrícola en Bolivia, eleme

In [27]:
len(useful_news_for_maiz_exports)

16

In [ ]:
# now the same but load the csv cultivos
import pandas as pd
df = pd.read_csv('base_data/cultivos.csv')
df

cultivos = df.to_dict('records')
cultivos

useful_news_for_cultivos = []
for item in cultivos:
	title = item.get('title')
	# the summary is either the attribute summary if not None or the first 200 characters of 'text'
	summary = item.get('summary') or item.get('text')[:200]
	
	completion = client.chat.completions.create(
		model="gpt-4-turbo",
		messages=[
			{"role": "system", "content": system_prompt},
			{
				"role": "user",
				"content": user_example
			},
			{
				"role": "assistant",
				"content": assistant_example
			},
			{
				"role": "user",
				"content": f"<news>{title}\n{summary}</news>"
			}
		]
	)
	try:
		completion = str(completion.choices[0].message.content)
		print(completion)
		if 'si.' in completion.lower():
			item['insight'] = completion
			useful_news_for_cultivos.append(item)
			print('si')
	except:
		continue

len(useful_news_for_cultivos)

In [28]:
# save json for maiz exports as news1.json {news:useful_news}
with open('news1.json', 'w') as f:
	json.dump({'news':useful_news_for_maiz_exports}, f)

In [29]:
# retrieve the name of the source countries in the list of useful news
countries = []
for item in useful_news_for_maiz_exports:
	countries.append(item.get('source_country'))

In [30]:
# print all the unq countries
countries = list(set(countries))
countries

['bo', 'cu', 'es', 'ar', 'pe']

In [32]:
# countries locations [lat, lon]
countries_locations = {
	'ar':[ -34.9964963, -64.9672817],
	'uy':[-32.522779, -55.765835],
	'br':[-14.235004, -51.92528],
	'py':[-23.442503, -58.443832],
	'bo':[-16.290154, -63.588653],
	'cl':[-35.675147, -71.542969],
	'pe':[-9.189967, -75.015152],
	'co':[4.570868, -74.297333],
	'ec':[-1.831239, -78.183406],
	've':[6.42375, -66.58973],
	'gt':[15.783471, -90.230759],
	'hn':[15.199999, -86.241905],
	'sv':[13.794185, -88.89653],
	'ni':[12.865416, -85.207229],
	'cr':[9.748917, -83.753428],
	'pa':[8.537981, -80.782127],
	'cu':[21.521757, -77.781167],
	'do':[18.735693, -70.162651],
	'pr':[18.220833, -66.590149],
	'mx':[23.634501, -102.552784],
	'us':[37.09024, -95.712891],
	'es':[40.463667, -3.74922],
	'fr':[46.603354, 1.888334],
	'it':[41.87194, 12.56738],
	'de':[51.165691, 10.451526],
	'jp':[36.204824, 138.252924],
	'cn':[35.86166, 104.195397],
	'ru':[61.52401, 105.318756],
}

In [33]:
# add lat, lon to the items and then save again as news.json
for item in useful_news_for_maiz_exports:
	item['lat'] = countries_locations.get(item.get('source_country'))[0]
	item['lon'] = countries_locations.get(item.get('source_country'))[1]

In [34]:
with open('news.json', 'w') as f:
	json.dump({'news':useful_news_for_maiz_exports}, f)